In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [9]:
from talus_data_analysis.plot import histogram
from talus_data_analysis.elib import Elib
from talus_data_analysis.load import read_df_from_s3
from talus_data_analysis.save import write_df_to_s3
from dotenv import load_dotenv
import tempfile
import sqlite3
import math
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [4]:
load_dotenv()

True

In [5]:
gauth = GoogleAuth(settings_file="../settings.yaml")

In [53]:
ENCYCLOPEDIA_BUCKET = "talus-data-pipeline-encyclopedia-bucket"
DATA_FOLDER = "../data/210525_Gryder"
S3_FOLDER = "wide/210525_Gryder"
PROJECT_NAME = "Gryder Contract"

ELIB_FILE = "RESULTS-quant.elib"
peptide_protein_file = "peptide_proteins_results.csv"
peptide_protein_norm_output = "peptide_proteins_normalized.csv"
msstats_groupcompare_output = "msstats_groupcompare.csv"
comparison_matrix_file = "comparison_matrix.csv"

In [163]:
peptide_df = read_df_from_s3(bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/RESULTS-quant.elib.peptides.txt", inputformat="txt")
peptide_df = peptide_df.drop("numFragments", axis=1)
# make sure there is one protein per column
peptide_df = peptide_df.drop("Protein", axis=1).join(peptide_df["Protein"].str.split(";", expand=True).stack().reset_index(drop=True, level=1).rename("Protein"))
# melt the dataframe so that each source file is in a seperate row instead of column
peptide_df = peptide_df.melt(id_vars=["Peptide", "Protein"], var_name="SourceFile", value_name="TotalIntensity")
peptide_df["Run"] = peptide_df["SourceFile"].apply(lambda x: x.split(".")[0].split("_")[-1])

In [164]:
peptide_df.tail()

,Peptide,Protein,SourceFile,TotalIntensity,Run
413023,YYVTIIDAPGHR,sp|P68104|EF1A1_HUMAN,210525_talus_32.mzML,26134348.0,32
413024,YYVTIIDAPGHRDFIK,sp|Q5VTE0|EF1A3_HUMAN,210525_talus_32.mzML,1922279.9,32
413025,YYVTIIDAPGHRDFIK,sp|P68104|EF1A1_HUMAN,210525_talus_32.mzML,1922279.9,32
413026,YYYAVVDC[+57.021464]DSPETASK,sp|Q9H501|ESF1_HUMAN,210525_talus_32.mzML,0.0,32
413027,YYYVPADFVEYEK,sp|Q93100|KPBB_HUMAN,210525_talus_32.mzML,2933829.2,32


## Template DF

In [165]:
sample_df = pd.read_excel("~/Downloads/MS_samples_May2021.xlsx")
sample_df = sample_df[(sample_df["Project"] == PROJECT_NAME) & (sample_df["Sample Description"] != "Pooled Sample")]
sample_df["Run"] = sample_df["Run ID"].apply(lambda x: x.split("_")[-1])
sample_df = sample_df.rename(columns={"Sample Name": "BioReplicate", "Sample Description": "Condition"})
sample_df = sample_df[["Run", "BioReplicate", "Condition", "Comparison"]]

In [166]:
sample_df

,Run,BioReplicate,Condition,Comparison
21,15,Gryder-1,DMSO - 24h,Control
22,16,Gryder-2,R1881 - 24h,DMSO - 24h
23,17,Gryder-3,BG15-n - 24h,DMSO - 24h
24,18,Gryder-4,ARN-505 - 24h,DMSO - 24h
25,19,Gryder-5,Biclutimide - 24h,DMSO - 24h
26,20,Gryder-6,BG-15a - 24h,DMSO - 24h
33,27,Gryder-7,DMSO - 48h,Control
34,28,Gryder-8,R1881 - 48h,DMSO - 48h
35,29,Gryder-9,BG15-n - 48h,DMSO - 48h
36,30,Gryder-10,ARN-505 - 48h,DMSO - 48h


In [167]:
msstats_df = pd.merge(peptide_df, sample_df, how="right", on="Run")

In [168]:
## Add a few required columns and rename header to match MSstats convention
msstats_df = msstats_df.drop(["Run", "Comparison"], axis=1)
msstats_df["PrecursorCharge"] = 2
msstats_df["IsotopeLabelType"] = "L"
msstats_df["FragmentIon"] = "y0"
msstats_df["ProductCharge"] = "1"
msstats_df = msstats_df.rename(columns={"Peptide": "PeptideSequence",
                                        "Protein": "ProteinName",
                                        "SourceFile": "Run",
                                        "TotalIntensity": "Intensity"})
msstats_df = msstats_df.drop_duplicates()
msstats_df = msstats_df.dropna(subset=["PeptideSequence", "ProteinName"])

In [169]:
msstats_df = msstats_df.sort_values(by=["PeptideSequence", "Intensity"]).reset_index(drop=True)

In [182]:
msstats_df.tail()

,PeptideSequence,ProteinName,Run,Intensity,BioReplicate,Condition,PrecursorCharge,IsotopeLabelType,FragmentIon,ProductCharge
412869,YYYVPADFVEYEK,sp|Q93100|KPBB_HUMAN,210525_talus_31.mzML,1228683.4,Gryder-11,Biclutimide - 48h,2,L,y0,1
412870,YYYVPADFVEYEK,sp|Q93100|KPBB_HUMAN,210525_talus_15.mzML,1389879.6,Gryder-1,DMSO - 24h,2,L,y0,1
412871,YYYVPADFVEYEK,sp|Q93100|KPBB_HUMAN,210525_talus_19.mzML,2438354.0,Gryder-5,Biclutimide - 24h,2,L,y0,1
412872,YYYVPADFVEYEK,sp|Q93100|KPBB_HUMAN,210525_talus_18.mzML,2744788.5,Gryder-4,ARN-505 - 24h,2,L,y0,1
412873,YYYVPADFVEYEK,sp|Q93100|KPBB_HUMAN,210525_talus_32.mzML,2933829.2,Gryder-12,BG-15a - 48h,2,L,y0,1


In [171]:
msstats_df.to_csv(f"{DATA_FOLDER}/{peptide_protein_file}")

In [172]:
write_df_to_s3(dataframe=msstats_df, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_file.replace('.csv', '.parquet')}", outputformat="parquet")

In [173]:
def get_comparison_matrix(df, filter_target_func=lambda x:x, leave_out=set()):
    df = df.sort_values(by="Condition")
    dmso_map = {condition: dmso for (condition, dmso) in zip(df["Condition"], df["Comparison"])}
    
    comp_lol = []
    targets = sorted(dmso_map.keys())
    targets = [t for t in targets if t not in leave_out]
    comp_df_index = []
    for i, s in enumerate(targets):
        comp_list = [0 for t in targets]
        if dmso_map[s] in dmso_map:
            # make dmso 0
            comp_list[targets.index(dmso_map[s])] = -1
            # make target itself 1
            comp_list[i] = 1

            comp_lol.append(comp_list)
            comp_df_index.append(f"{filter_target_func(s)}/{dmso_map[s]}")

    comp_df = pd.DataFrame(comp_lol)
    comp_df.index = comp_df_index
    
    return comp_df

In [174]:
comp_matrix = get_comparison_matrix(df=sample_df, 
                                    filter_target_func=lambda x: x.split(" - ")[0],
                                    leave_out=set(["R1881 - 24h"])
                                   )

In [175]:
comp_matrix.to_csv(f"{DATA_FOLDER}/{comparison_matrix_file}")

# Run R Script (MSStats) ...

## Write msstats normalized peptide protein df to s3

In [176]:
msstats_df_norm = pd.read_csv(f"{DATA_FOLDER}/{peptide_protein_norm_output}")

In [177]:
msstats_df_norm

,PROTEIN,PEPTIDE,TRANSITION,FEATURE,LABEL,GROUP_ORIGINAL,SUBJECT_ORIGINAL,RUN,GROUP,SUBJECT,INTENSITY,SUBJECT_NESTED,ABUNDANCE,FRACTION,originalRUN,censored
0,sp|A0A075B767|PAL4H_HUMAN,HTGSGILSMVNAGPNTNGSQLFIC[+57.021464]TAK_2,y0_1,HTGSGILSMVNAGPNTNGSQLFIC[+57.021464]TAK_2_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,5,1.078618e+06,1.5,19.584472,1,210525_talus_18.mzML,False
1,sp|A0A0B4J1U3|LV136_HUMAN,LLIYYDDLLPSGVSDRFSGSK_2,y0_1,LLIYYDDLLPSGVSDRFSGSK_2_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,5,6.375610e+04,1.5,15.503995,1,210525_talus_18.mzML,False
2,sp|A0A0K0K1C4|TVB27_HUMAN,MGPQLLGYVVLC[+57.021464]LLGAGPLEAQVTQNPR_2,y0_1,MGPQLLGYVVLC[+57.021464]LLGAGPLEAQVTQNPR_2_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,5,1.253390e+06,1.5,19.801124,1,210525_talus_18.mzML,False
3,sp|A0A1B0GTL2|CT204_HUMAN,AYSPAC[+57.021464]SVPDVLR_2,y0_1,AYSPAC[+57.021464]SVPDVLR_2_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,5,3.221410e+06,1.5,21.162981,1,210525_talus_18.mzML,False
4,sp|A0AV96|RBM47_HUMAN,IPTAGIYGASYVPFAAPATATIATLQK_2,y0_1,IPTAGIYGASYVPFAAPATATIATLQK_2_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,5,1.221128e+05,1.5,16.441575,1,210525_talus_18.mzML,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339488,sp|Q9Y6Y8|S23IP_HUMAN,SAWQTLNEFAR_2,y0_1,SAWQTLNEFAR_2_y0_1,L,R1881 - 48h,Gryder-8,11,11,8,2.652112e+06,11.8,21.014005,1,210525_talus_28.mzML,False
339489,sp|Q9Y6Y8|S23IP_HUMAN,SIIEC[+57.021464]VDDFRVVSLK_2,y0_1,SIIEC[+57.021464]VDDFRVVSLK_2_y0_1,L,R1881 - 48h,Gryder-8,11,11,8,3.451473e+05,11.8,18.072148,1,210525_talus_28.mzML,False
339490,sp|Q9Y6Y8|S23IP_HUMAN,TMNISPEQPQH_2,y0_1,TMNISPEQPQH_2_y0_1,L,R1881 - 48h,Gryder-8,11,11,8,2.779201e+06,11.8,21.081534,1,210525_talus_28.mzML,False
339491,sp|Q9Y6Y8|S23IP_HUMAN,VIVQFQPSSVPDEWGTTQDGQTRPR_2,y0_1,VIVQFQPSSVPDEWGTTQDGQTRPR_2_y0_1,L,R1881 - 48h,Gryder-8,11,11,8,2.844630e+07,11.8,24.437032,1,210525_talus_28.mzML,False


In [178]:
write_df_to_s3(dataframe=msstats_df_norm, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_norm_output.replace('.csv', '.parquet')}", outputformat="parquet")

## Write msstats groupcompare df to s3

In [179]:
msstats_groupcompare = pd.read_csv(f"{DATA_FOLDER}/{msstats_groupcompare_output}")

In [180]:
msstats_groupcompare

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
0,sp|A0A075B767|PAL4H_HUMAN,ARN-505/DMSO - 24h,0.395907,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000
1,sp|A0A0B4J1U3|LV136_HUMAN,ARN-505/DMSO - 24h,-4.099830,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000
2,sp|A0A0K0K1C4|TVB27_HUMAN,ARN-505/DMSO - 24h,0.337720,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000
3,sp|A0A1B0GTL2|CT204_HUMAN,ARN-505/DMSO - 24h,-0.435512,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000
4,sp|A0AV96|RBM47_HUMAN,ARN-505/DMSO - 24h,1.415634,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
46570,sp|Q9Y6X4|F169A_HUMAN,R1881/DMSO - 48h,0.278825,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000
46571,sp|Q9Y6X8|ZHX2_HUMAN,R1881/DMSO - 48h,0.578473,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000
46572,sp|Q9Y6X9|MORC2_HUMAN,R1881/DMSO - 48h,0.415727,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000
46573,sp|Q9Y6Y0|NS1BP_HUMAN,R1881/DMSO - 48h,0.182987,NaN,NaN,0.0,NaN,NaN,NaN,0.400000,0.400000


In [181]:
write_df_to_s3(dataframe=msstats_groupcompare, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{msstats_groupcompare_output.replace('.csv', '.parquet')}", outputformat="parquet")